# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load the csv
weather_data_to_load = "data/weather_data.csv"
weather_data_df = pd.read_csv(weather_data_to_load)
weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tasiilaq,65.6145,-37.6368,42.96,71,16,1.14,GL,1660608749
1,rio gallegos,-51.6226,-69.2181,35.65,75,40,12.66,AR,1660609205
2,castro,-24.7911,-50.0119,60.21,68,0,5.59,BR,1660608800
3,rikitea,-23.1203,-134.9692,73.36,76,1,14.54,PF,1660608409
4,punta arenas,-53.1500,-70.9167,37.51,81,20,16.11,CL,1660608694
...,...,...,...,...,...,...,...,...,...
552,vanderhoof,54.0166,-124.0198,70.84,52,100,2.71,CA,1660609426
553,nome,64.5011,-165.4064,47.16,62,75,9.22,US,1660609427
554,lingyuan,41.2400,119.4011,69.04,65,0,7.58,CN,1660609428
555,langcangan,8.4794,123.7558,79.92,76,84,1.57,PH,1660609428


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Declaring parameter variables
locations = weather_data_df[["Lat","Lng"]]
weights = weather_data_df["Humidity"]

In [4]:
# Plot
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = weights, dissipating = False, max_intensity = 100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Filtering ideal cities based on weather criteria
hotel_df = weather_data_df.loc[(weather_data_df["Max Temp"] < 80) & (weather_data_df["Max Temp"] > 70) \
                                   & (weather_data_df["Wind Speed"] < 10) & (weather_data_df["Cloudiness"] == 0)]
hotel_df.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,bethel,41.3712,-73.4140,75.29,59,0,4.61,US,1660608825
58,gat,31.6100,34.7642,77.81,100,0,4.90,IL,1660608695
64,pacific grove,36.6177,-121.9166,77.23,83,0,9.22,US,1660608752
123,tukrah,32.5341,20.5791,77.99,68,0,8.28,LY,1660609248
139,corinto,37.9407,22.9573,72.90,70,0,4.34,GR,1660609253
157,manicore,-5.8092,-61.3003,78.75,89,0,3.44,BR,1660609260
168,mangaratiba,-22.9597,-44.0406,74.26,53,0,8.25,BR,1660609265
182,maarianhamina,60.0973,19.9348,71.46,78,0,8.05,AX,1660609269
271,dongsheng,39.8161,109.9776,70.48,48,0,5.82,CN,1660609305
291,freeport,40.6576,-73.5832,76.17,60,0,5.75,US,1660609314


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Variables declaration
first_hotels_name_ls = []
first_hotels_add_ls = []
target_search = "Hotel"
radius = 5000
target_location = hotel_df[["Lat","Lng"]].values.tolist()
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""

C:\Users\Nhan Tran\AppData\Local\Temp\ipykernel_10972\3175996817.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
C:\Users\Nhan Tran\AppData\Local\Temp\ipykernel_10972\3175996817.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Address"] = ""


In [7]:
# Search for the first hotel in each coordinate
for loc in target_location:
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    my_params  = {
    "location": f'{loc[0]}, {loc[1]}', 
    "keyword": target_search,
    "radius": radius,
    "key": g_key
    }
    first_hotels = requests.get(base_url,params = my_params).json()
    
    try:
        first_hotels_name_ls.append(first_hotels["results"][0]["name"])
        first_hotels_add_ls.append(first_hotels["results"][0]["vicinity"]) 
    
    except IndexError:
        first_hotels_name_ls.append(None)
        first_hotels_add_ls.append(None)
        print(f'Hotel not found. Skipping...')
        
    time.sleep(1)

Hotel not found. Skipping...
Hotel not found. Skipping...


In [8]:
# Adding list data to hotel 
hotel_df["Hotel Name"] = first_hotels_name_ls
hotel_df["Hotel Address"] = first_hotels_add_ls


C:\Users\Nhan Tran\AppData\Local\Temp\ipykernel_10972\2826274161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = first_hotels_name_ls
C:\Users\Nhan Tran\AppData\Local\Temp\ipykernel_10972\2826274161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Address"] = first_hotels_add_ls


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
hotel_weights = hotel_df["Humidity"]

In [11]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = hotel_weights, dissipating = False, max_intensity = 100, point_radius = 1)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(target_location,info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))